# IBM DS Capstone Assignment

<strong>Import all libraries, install any that are not installed, and set pd options.</strong>

In [2]:
# !pip install bs4
!pip install folium
# !pip install scikit-learn
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import sklearn
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01


<strong>Scrape table, pass to df and print df shape.</strong>

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
soup=BeautifulSoup(r.content,'html5lib')

table_contents=[]
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
tor_df=pd.DataFrame(table_contents)
tor_df['Borough']=tor_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
tor_df.shape

(103, 3)

<strong>Import df from csv file. Requests with geocoder took too long.</strong>

In [8]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_c0a055bfb9f342ba832a39c186e8bf1b = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_c0a055bfb9f342ba832a39c186e8bf1b = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_c0a055bfb9f342ba832a39c186e8bf1b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='dF18Sc8aeBBeiZTux0bonS8hJWzCA09le9ysrdoGNt5J',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_c0a055bfb9f342ba832a39c186e8bf1b)

body = client_c0a055bfb9f342ba832a39c186e8bf1b.get_object(Bucket='dscapstone-donotdelete-pr-lljtteeeoruzbn',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<strong>Pass lats and longs from df_data_1 to tor_df based on postal code.</strong>

In [9]:
for index, row in tor_df.iterrows():
    postal_code=row['PostalCode']
    if str(postal_code) in df_data_1['Postal Code'].values:
        idx=df_data_1.index
        tor_df['Latitude']=df_data_1['Latitude'][idx]
        tor_df['Longitude']=df_data_1['Longitude'][idx]
tor_df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills North,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


<strong>Set map starting point, initialize KMeans object and map clusters.</strong>

In [10]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

kclusters=5
tor_df_cluster=tor_df.drop('PostalCode', 1)
tor_df_cluster=tor_df_cluster.drop('Neighborhood', 1)
tor_df_cluster=tor_df_cluster.drop('Borough', 1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(tor_df_cluster)
tor_df_cluster.insert(0, 'Cluster Labels', kmeans.labels_)

tor_map = folium.Map(location=[latitude, longitude], zoom_start=10)

x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

for lat, lon, cluster in zip(tor_df_cluster['Latitude'],tor_df_cluster['Longitude'],tor_df_cluster['Cluster Labels']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tor_map)
tor_map